In [1]:
from library.experiments import *
from library.objective_function import *
from library.optimiser import *

In [2]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)
ad = adam()
optParas = {
         'x0': torch.tensor([16.23, 23.023], requires_grad=True),
         'alpha': 0.1,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-11, 
         'max_iter': 2000,
         'tol': 1e-6,              
         'verbose': True,
         'record': False }
ad.set_parameters(optParas)
exp.set_optimizer(ad)
exp.do()



*******starting optimisation from intitial point:  [16.23  23.023]
total evaluatios = 201
gradient at stop position = tensor([-0.0001,  0.0001]),
modified graident = tensor([-2.1003e-06, -8.8239e-06])
found minimum position = [15.999424 22.999178], found minimum = 19.61959457397461
Result:  local minimum
found minimum: 19.61959457397461, minimum position: [15.999424 22.999178], evals: 201


('local minimum', tensor(19.6196, grad_fn=<AddBackward0>), 201)

In [3]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)
ln = line_search()
optParas = {
    'x0': torch.tensor([16.23, 23.023], requires_grad=True),
    'alpha': 1,
    'beta': 0.1, 
    'max_iter': 1000,
    'tol': 1e-3,              
    'verbose': True,
    'record': False
}
ln.set_parameters(optParas)
exp.set_optimizer(ln)
exp.do()


*******starting optimisation from intitial point:  [16.23  23.023]
total evaluatios = 41
gradient at stop position = [0.00045202 0.00023452]
found minimum position = [15.999435 22.99918 ], found minimum = 19.61959457397461
Result:  local minimum
found minimum: 19.61959457397461, minimum position: [15.999435 22.99918 ], evals: 41


('local minimum', tensor(19.6196, grad_fn=<AddBackward0>), 41)

In [17]:
class cma_es(adjust_optimizer):
    def __init__(self, dim=2):
        self.dim = dim
        paras = {'x0': torch.zeros((dim,)),
                 'std': torch.ones((dim,)) * 3, 
                 'tol': 1e-5, 
                 'adjust_func': do_nothing(), 
                 'record': False, 
                 'verbose': False}
        self.set_parameters(paras)
    def set_parameters(self, paras):
        self.paras = paras
        self.x0 = paras['x0'] 
        self.std = paras['std']
        self.tol = paras['tol']
        self.adjust_func = paras['adjust_func']
        self.max_iter = 400 if 'max_iter' not in paras.keys() else paras['max_iter']
        # set none to use default value 
        self.cluster_size = None if 'cluster_size' not in paras.keys() else paras['cluster_size']
        self.survival_size = None if 'survival_size' not in paras.keys() else paras['survival_size']
        self.record = True if 'record' not in paras.keys() else paras['record']
        self.verbose = True if 'verbose' not in paras.keys() else paras['verbose']
   
    def __update_mean(self, x):
        return (self.weights @ x).reshape(self.dim, 1)
    
    def __update_ps(self):
        return (1 - self.cs) * self.ps + torch.sqrt(self.cs * (2 - self.cs) * self.MU_EFF) * self.invsqrtC @ (self.mean - self.mean_old) / self.sigma 
    
    def __update_pc(self):
        hsig = (torch.norm(self.ps) / torch.sqrt(1 - (1 - self.cs)**(2 * self.iter_/self.lambda_)) / self.chiN < 1.4 + 2/(self.dim + 1)).int()
        return (1 - self.cc) * self.pc + hsig * torch.sqrt(self.cc * (2 - self.cc) * self.MU_EFF) * (self.mean - self.mean_old) / self.sigma
    
    def __update_C(self):
        hsig = (torch.norm(self.ps) / torch.sqrt(1 - (1 - cs)**(2 * iter_/lambda_)) / chiN < (1.4 + 2/(dim + 1))).int()
        artmp = (1 / self.sigma) * (x - mean_old.reshape(1, dim))
        return (1 - self.c1 - self.cmu) * self.C + self.c1 * (self.pc * self.pc.T + (1 - self.hsig) * self.cc * (2 - self.cc) * C) + self.cmu * artmp.T @ torch.diag(self.weights) @ artmp
    
    def __update_sigma(self):
        return self.sigma * torch.exp((self.cs / self.damps) * (torch.norm(self.ps)/ self.chiN - 1))
    
    def __is_not_moving(arg, val, pre_arg, pre_val, tol):
        dis_arg = torch.norm(arg - pre_arg, dim=1).mean()
        dis_val = torch.abs(val - pre_val).mean()
        return (dis_arg < tol and dis_val < tol) 

    def __step(self):
        for i in range(lambda_):
            x[i] = (mean + sigma * B @ torch.diag(D) @ torch.randn(dim, 1)).squeeze()
            x_old[i] = x[i]
            x[i], f[i], inner_stats[i] = self.adjust_func.adjust(x[i].clone().detach().requires_grad_(True), obj)
            eval_ += inner_stats[i]['evals']
            iter_eval[i] = inner_stats[i]['evals']
        # sort the value and positions of solutions 
        idx = torch.argsort(f.detach())
        x_ascending = x[idx]

        # update the parameter for next iteration
        mean_old = mean
        mean = self__update_mean(x_ascending[:mu])
        self.__update_ps()
        self.__update_pc()
        self.__update_sigma()
        self.__update_C()
        C = torch.triu(C) + torch.triu(C, 1).T
        D, B = torch.eig(C, eigenvectors=True)
        D = torch.sqrt(D[:,0])
        invsqrtC = B @ torch.diag(D**-1) @ B

        # record data during process for post analysis
        if self.record:
            self.stats['inner'].append(inner_stats.clone().detach().numpy())
            self.stats['arg'].append(x_ascending.detach().numpy())
            self.stats['val'].append(f[idx].detach().numpy())
            self.stats['mean'].append(mean.detach().numpy())
            self.stats['std'].append((sigma * B @ np.diag(D)).detach().numpy())
            self.stats['evals_per_iter'].append(iter_eval.clone().detach().numpy())
            self.stats['x_adjust'].append(np.vstack((x_old.T.clone().detach().numpy(), x.T.clone().detach().numpy())))
        # record best solution    
        if best_val > f[idx[0]]:
            best_val = f[idx[0]]
            best_arg = x_ascending[0]
        return x_ascending, f[idx]
        
    def __setup():
                # User defined input parameters 
        self.sigma = 0.3
        self.D = self.std / sigma
        self.mean = self.x0.reshape(dim, 1)
        # the size of solutions group
        self.lambda_ = 4 + int(3 * np.log(dim)) if self.cluster_size == None else self.cluster_size  
        # only best "mu" solutions are used to generate iterations
        self.mu = int(lambda_ / 2) if self.survival_size == None else self.survival_size
        # used to combine best "mu" solutions                                               
        weights = np.log(mu + 1/2) - torch.log(torch.arange(mu, dtype=torch.float) + 1) 
        self.weights = (weights / torch.sum(weights)).float()    
        self.MU_EFF = 1 / torch.sum(weights**2) 

        # Strategy parameter setting: Adaptation
        # time constant for cumulation for C
        self.cc = (4 + self.mueff / self.dim) / (self.dim + 4 + 2 * self.MU_EFF / self.dim)  
        # t-const for cumulation for sigma control
        self.cs = (self.MU_EFF + 2) / (self.dim + self.MU_EFF + 5)  
        # learning rate for rank-one update of C
        self.c1 = 2 / ((self.dim + 1.3)**2 + self.MU_EFF)    
        # and for rank-mu update
        self.cmu = min(1 - self.c1, 2 * (sefl.MU_EFF - 2 + 1 / self.MU_EFF) / ((self.dim + 2)**2 + self.MU_EFF))  
        # damping for sigma, usually close to 1  
        self.damps = 1 + 2 * max(0, np.sqrt((self.MU_EFF - 1)/( self.dim + 1)) - 1) + self.cs                                                                 

        # Initialize dynamic (internal) strategy parameters and constants
        # evolution paths for C and sigma
        self.pc = torch.zeros((self.dim, 1))     
        self.ps = torch.zeros((self.dim, 1)) 
        # B defines the coordinate system
        self.B = torch.eye(int(self.dim))       
        # covariance matrix C
        self.C = self.B * torch.diag(self.D**2) * self.B.T 
        # C^-1/2 
        self.invsqrtC = self.B * torch.diag(self.D**-1) * self.B.T   
        # expectation of ||N(0,I)|| == norm(randn(N,1)) 
        self.chiN = self.dim**0.5 * (1 - 1/(4 * self.dim) + 1 / (21 * self.dim**2))  

        # --------------------  Initialization --------------------------------  
        x, x_old, f = torch.zeros((lambda_, dim)), torch.zeros((lambda_, dim)), torch.zeros((lambda_,))
        self.stats = {}
        inner_stats = {}
        self.stats['inner'] = []
        self.stats['val'], stats['arg'] = [], []
        self.stats['x_adjust'] = []
        iter_eval, stats['evals_per_iter'] = torch.zeros((lambda_, )), []
        inner_stats = [{}] * lambda_
        self.stats['mean'], stats['std'] = [], []
        self.stats['status'] = None
        iter_, eval_ = 0, 0
        
    def optimise(self, obj):
        '''
        @param obj: objective function class instance
        return arg: found minimum arguments
               val: found minimum value
               stats: collection of recorded statistics for post-analysis
        '''                  
        if self.verbose:
            print("\n\n*******starting optimisation from intitial mean: ", self.x0.squeeze().detach().numpy())
        self.__setup()
        
        # initial data in record
        for i in range(lambda_):
            x[i,:] = (mean + torch.randn(dim, 1)).squeeze()
            f[i] = obj.func(x[i])
        idx = torch.argsort(f.detach())
        x_ascending = x[idx]
        if self.record:
            stats['inner'].append(inner_stats.detach().numpy())
            stats['arg'].append(x_ascending.detach().numpy())
            stats['val'].append(f[idx].detach().numpy())
            stats['mean'].append(mean.detach().numpy())
            stats['std'].append(sigma * B @ torch.diag(D))
            stats['evals_per_iter'].append(torch.ones((lambda_,)).detach().numpy())
            stats['x_adjust'].append(np.vstack((x.T.clone().detach().numpy(), x.T.clone().detach().numpy())))
        arg = x_ascending
        val = f[idx]
        
        pre_arg = x_ascending
        pre_val = f[idx]
        self.best_val = 1e4
        self.best_arg = None
        # optimise by iterations
        try:
            while iter_ < self.max_iter:
                iter_ += 1
                arg, val = self.__step()
                # check the stop condition
                if torch.max(D) > (torch.min(D) * 1e6):
                    stats['status'] = 'diverge'
                    print('diverge, concentrate in low dimension manifold')
                    break
                if self.__is_not_moving(arg, val, pre_arg, pre_val, self.tol) :
                    break
                pre_arg = arg
                pre_val = val
        except np.linalg.LinAlgError as err:
            stats['status'] = 'diverge'
            print('diverge, raise LinAlgError!')
        finally:
            if self.verbose:
                print('eigenvalue of variance = {}'.format(D))
                print('total iterations = {}, total evaluatios = {}'.format(iter_, eval_))
                print('found minimum position = {}, found minimum = {}'.format(best_arg.detach().numpy(), best_val.detach().numpy()))

        # carry statistics info before quit
        if self.record:
            stats['arg'] = np.array(stats['arg'])
            stats['val'] = np.array(stats['val'])
            stats['mean'] = np.array(stats['mean'])
            stats['std'] = np.array(stats['std'])
            stats['evals_per_iter'] = np.array(stats['evals_per_iter'])
            stats['x_adjust'] = np.array(stats['x_adjust'])
        stats['evals'] = eval_
        return best_arg, best_val, stats

In [29]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)
opt = cma_es(dim=2)
optParas ={'x0': torch.tensor([196.23, 123.023], requires_grad=True),
           'std': torch.ones((2,)) * 3, 
           'tol': 1e-3, 
           'adjust_func': do_nothing(), 
           'record': False, 
           'verbose': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
exp.do()



*******starting optimisation from intitial mean:  [196.23  123.023]
eigenvalue of variance = tensor([0.2339, 0.1790], grad_fn=<SqrtBackward>)
total iterations = 95, total evaluatios = 570
found minimum position = [197.00003 123.00017], found minimum = 20.0
Result:  local minimum
found minimum: 20.0, minimum position: [197.00003 123.00017], evals: 570


('local minimum', tensor(20., grad_fn=<SelectBackward>), 570)

In [25]:
# init and setup one experiment
exp = single_experiment()
# One experiment: setup objective function
ak = ackley()
exp.set_objective(ak)

inner_opt = line_search()
innerOptParas = {
   'x0': None,
    'alpha': 1,
    'beta': 0.1, 
    'max_iter': 100,
    'tol': 1e-1,              
    'verbose': False,
    'record': False
}
inner_opt.set_parameters(innerOptParas)

opt = cma_es()
optParas ={'x0': torch.tensor([196.23, 123.023], requires_grad=True),
           'std': torch.ones((2,)) * 30, 
           'tol': 1e-3, 
           'adjust_func': inner_opt, 
           'record': False, 
           'verbose': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
exp.do()



*******starting optimisation from intitial mean:  tensor([196.2300, 123.0230], grad_fn=<SqueezeBackward0>)
eigenvalue of variance = tensor([ 8.1473, 12.1323], grad_fn=<SqrtBackward>)
total iterations = 35, total evaluatios = 12136
found minimum position = tensor([ 4.1871e-07, -1.2184e-07], grad_fn=<SelectBackward>), found minimum = 9.5367431640625e-07
Result:  global minimum
found minimum: 9.5367431640625e-07, minimum position: [ 4.1870697e-07 -1.2184466e-07], evals: 12136


('global minimum', tensor(9.5367e-07, grad_fn=<SelectBackward>), 12136)

In [30]:
exp = single_experiment()
# setup objective function
ak = ackley()
exp.set_objective(ak)

inner_opt = adam()
optParas = {
         'x0': None,
         'alpha': 1,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-11, 
         'max_iter': 100,
         'tol': 1e-1,              
         'verbose': False,
         'record': False }
inner_opt.set_parameters(optParas)

opt = cma_es()
optParas ={'x0': torch.tensor([196.23, 123.023], requires_grad=True),
           'std': torch.ones((2,)) * 30, 
           'tol': 1e-3, 
           'adjust_func': inner_opt, 
           'record': False, 
           'verbose': True}
opt.set_parameters(optParas)
exp.set_optimizer(opt)
exp.do()



*******starting optimisation from intitial mean:  tensor([196.2300, 123.0230], grad_fn=<SqueezeBackward0>)
eigenvalue of variance = tensor([0.0018, 0.0028], grad_fn=<SqrtBackward>)
total iterations = 400, total evaluatios = 186400
found minimum position = tensor([14.0245, -5.0542], grad_fn=<SelectBackward>), found minimum = 17.66341209411621
Result:  local minimum
found minimum: 17.66341209411621, minimum position: [14.024487 -5.054191], evals: 186400


('local minimum', tensor(17.6634, grad_fn=<SelectBackward>), 186400)